In [1]:
import cv2 as cv
import math
import numpy as np
from pathlib import Path

def load_img(path):
    return cv.imread(str(path.resolve()), cv.IMREAD_GRAYSCALE)
    
def convert2binary(img):
    _, binary = cv.threshold(img, 127, 1, cv.THRESH_BINARY_INV)
    return binary

def left_corner_slice(img):
    x = 0
    y = 0
    w = 200
    h = 200
    return img[y:y+h, x:x+w]

def right_corner_slice(img):
    x = img.shape[1] - 200  #100px from the right
    y = 0                   #  0px from the top
    w = 200
    h = 200
    return img[y:y+h, x:x+w]

def find_corner(img):
    # first point
    # find y
    y_index = 0
    for y in img:
        if any(y):
            break
        y_index += 1
        
    # find x
    x_index = 0
    for x in img[y_index]:
        if x == 1:
            break
        x_index += 1
    
    #second point
    break_first_loop = False
    sec_x_index = 0
    for x in img[0]:
        if not break_first_loop:
            sec_y_index = 0
            for y in img:
                if y[sec_x_index] == 1:
                    break_first_loop = True
                    break
                sec_y_index += 1
        else:
            break
        sec_x_index += 1
        
    # determine CW or CCW
    if (x_index - sec_x_index) > (sec_y_index - y_index):
        # second point, CCW
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 2")
        return sec_x_index, sec_y_index
    elif (x_index - sec_x_index) == 0:
        # first point, both Xs are the same
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 1")
        return x_index, y_index
    elif (sec_y_index - y_index) == 0:
        # second point, both Ys are the same
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 2")
        return sec_x_index, sec_y_index
    else:            
        # first point, CW
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 1")
        return x_index, y_index

def mark_corner(img, center):
    small_radius = 2
    big_radius = 6
    color = (0,0,0) #black

    circled = cv.circle(img, center, small_radius, color)
    circled = cv.circle(img, center, big_radius, color)
    return circled

In [2]:
for f in Path("handwriting/").iterdir():
    file_ = str(f.resolve())
    if not str(f.name).startswith(".") and str(f.name).endswith(".jpg"):
        original = load_img(f)
        binary = convert2binary(original)
        
        # slice the corners, for convenient manual inspection
        right_corner_binary = right_corner_slice(binary)
        right_corner = right_corner_slice(original)
        left_corner_binary = left_corner_slice(binary)
        left_corner = left_corner_slice(original)
        
        # mark the left corner
        x, y = find_corner(left_corner_binary)
        left = (x, y)
        left_circled = mark_corner(left_corner, left)
        
        # mark the right corner
        x, y = find_corner(np.flip(right_corner_binary, 1))  # flip the image horizontally first
        right = (right_corner.shape[1]-x, y)  # compensate for the flipped value
#         print(f"corner: {right_center}")
        right_circled = mark_corner(right_corner, right)
        
        # show left and right corners in one window for manual decision
        combined_corners = np.hstack((left_circled, right_corner))
#         cv.imshow("combined_corners", combined_corners)
        



        # calculate the right point as if it were part of the original image
        # A is left corner, B is right corner
        A = left
        B = (original.shape[1]-200+right[0], right[1])
        
        # calculate rotation angle (make y-values equal), then rotate
        opp = B[1] - A[1]
        adj = B[0] - A[0]
        degrees = math.degrees(math.atan(opp/adj))
        print("degrees: ", degrees)
#         print(f"L: {left[0]}, {left[1]} | R: {right[0]}, {right[1]}")
        
        rotation_matrix = cv.getRotationMatrix2D((A[0], A[1]), degrees, 1)
        original_rotated = cv.warpAffine(original, rotation_matrix, (original.shape[1], original.shape[0]))
#         cv.imshow("original_rotated", original_rotated)

        h = original_rotated.shape[0]
        w = original_rotated.shape[1]
#             cropped = original[y:y+h, x:x+w]
        x = A[0]
        y = A[1]
        crop_height = h-A[1]
        crop_width = A[0]+B[0]
        cropped = original_rotated[y:crop_height, x:crop_width]
#             cv.imwrite(f"fine_cropped/{str(f.name)}", cropped)
        cv.imshow("cropped", cropped)
        
#         # decide to keep or reject
        k = cv.waitKey(0)
        if k == 27: #esc
            cv.destroyAllWindows()
            exit()
            break

        if k == ord("a"):
#             crop original and save
            h = original.shape[0]
            w = original.shape[1]
#             cropped = original[y:y+h, x:x+w]
            x = A[0]
            y = A[1]
            crop_height = h-A[1]
            crop_width = A[0]+B[0]
            cropped = original[y:crop_height, x:crop_width]
#             cv.imwrite(f"fine_cropped/{str(f.name)}", cropped)
            cv.imshow("cropped", cropped)
        if k == ord("r"):
#             reject, save original to need_to_fine_crop
#             cv.imwrite(f"need_to_fine_crop/{str(f.name)}", original)
            print(f.name)


degrees:  0.04986576592398655
degrees:  0.24803212815650258
degrees:  0.1497273059812821
degrees:  0.1497273059812821
degrees:  0.0
degrees:  0.049606723113913925
degrees:  0.049606723113913925
degrees:  0.643189849227418
degrees:  0.1996360545243788
degrees:  0.04990920298035432
degrees:  0.0
degrees:  0.0
degrees:  0.0
degrees:  0.09921337185647205
degrees:  0.04990920298035432
degrees:  0.1497273059812821
degrees:  0.0
degrees:  -0.04990920298035432
degrees:  0.049606723113913925
degrees:  0.1488198718569876
degrees:  -0.04995271577695466
degrees:  0.0
degrees:  -0.04990920298035432
degrees:  0.0
degrees:  0.0
degrees:  -0.099818330220591
degrees:  0.04995271577695466
degrees:  0.2997136368568732
degrees:  0.1996360545243788
degrees:  0.0
degrees:  0.14998860898745797
degrees:  0.19842614874611172
degrees:  -0.1488198718569876
degrees:  0.14959699560250583
degrees:  0.19825450119919608
degrees:  0.0
degrees:  0.09912754731167069
degrees:  0.14894883130382333
degrees:  -0.04960672311